In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle

In [ ]:
# Mount Drive to Google Colab
from google.colab import files
files.upload()

In [ ]:
# Connect to Kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d huangruichu/caltech101

In [ ]:
# Unzip the downloaded Dataset
!unzip /content/caltech101.zip

In [ ]:
pip install scipy==1.1.0

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import seaborn as sns
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
%matplotlib inline
import tensorflow as tf
import tensorflow_datasets as tfdb
import keras
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import scipy

In [ ]:
# Load the Training Dataset
def imread(path):
    img = scipy.misc.imread(path).astype(np.float)
    if len(img.shape) == 2:
        img = np.transpose(np.array([img, img, img]), (2, 0, 1))
    return img
    
path = '/content/Caltech101/Caltech101/train'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

categories = sorted(os.listdir(path))
ncategories = len(categories)
train = []
train_labels = []
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = scipy.misc.imresize(imread(fullpath), [128,128, 3])
            img = img.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            train.append(img) # NORMALIZE IMAGE 
            label_curr = i
            train_labels.append(label_curr)
        #iter = (iter+1)%10;
print ("Num train imgs: %d" % (len(train)))
print ("Num train labels: %d" % (len(train_labels)) )
print ("train catg=",ncategories)

In [ ]:
# Load the Test Dataset
path = '/content/Caltech101/Caltech101/test'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

categories = sorted(os.listdir(path))
ncategories = len(categories)
test = []
test_labels = []
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = scipy.misc.imresize(imread(fullpath), [128,128, 3])
            img = img.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            test.append(img) # NORMALIZE IMAGE 
            label_curr = i
            test_labels.append(label_curr)
        #iter = (iter+1)%10;
print ("Num test imgs: %d" % (len(test)))
print ("Num test labels: %d" % (len(test_labels)) )
print ("test catg=",ncategories)

In [ ]:
#Load the Validation Dataset
path = '/content/Caltech101/Caltech101/eval'
valid_exts = [".jpg", ".gif", ".png", ".jpeg"]
print ("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(path)), path))

categories = sorted(os.listdir(path))
ncategories = len(categories)
vald = []
vald_labels = []
# LOAD ALL IMAGES 
for i, category in enumerate(categories):
    iter = 0
    for f in os.listdir(path + "/" + category):
        if iter == 0:
            ext = os.path.splitext(f)[1]
            if ext.lower() not in valid_exts:
                continue
            fullpath = os.path.join(path + "/" + category, f)
            img = scipy.misc.imresize(imread(fullpath), [128,128, 3])
            img = img.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            vald.append(img) # NORMALIZE IMAGE 
            label_curr = i
            vald_labels.append(label_curr)
        #iter = (iter+1)%10;
print ("Num eval imgs: %d" % (len(vald)))
print ("Num eval labels: %d" % (len(vald_labels)) )
print ("eval catg= ",ncategories)

In [ ]:
X_train= np.array(train)
X_train= X_train/255.0
print(X_train.shape)

X_test= np.array(test)
X_test= X_test/255.0
print(X_test.shape)

X_vald= np.array(vald)
X_vald= X_vald/255.0
print(X_vald.shape)

In [ ]:
X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)
X_vald = np.stack(X_vald, axis=0)

In [ ]:
# One hot Encoding
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical( train_labels, num_classes = 102)
y_testHot = to_categorical( test_labels, num_classes = 102)
y_valdHot = to_categorical( vald_labels, num_classes = 102)

print(np.unique(train_labels))
print(np.unique(test_labels))
print(np.unique(vald_labels))

In [ ]:
#Downloading VGG 19 Model
base_model = keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(128,128, 3))

In [ ]:
last = base_model.get_layer('block5_pool').output
# Adding classification layers on top of it
x = BatchNormalization()
x = BatchNormalization()


x = Flatten()(last)

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x=  Dense(512, activation='relu')(x)
x= Dense(256, activation='relu')(x)
pred = Dense(102, activation='softmax')(x)

model =Model(base_model.input, pred)


In [ ]:
model.summary()

In [ ]:
# Compiling the Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1, momentum=0.9),
              metrics=['acc'])


In [ ]:
a = X_train
b = y_trainHot

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

In [ ]:
# Fitting the Model
datagen.fit(a)
batch_size = 128
history = model.fit(a,b, epochs=20, batch_size=128, validation_data=(X_vald,y_valdHot))

In [ ]:
# Model Evaluation
test_loss, test_acc = model.evaluate(X_test,y_testHot)
print("Test Loss: ", test_loss*100)
print("Test Accuracy: ", test_acc*100)

In [ ]:
# Plotting Model Accuracy
plt.plot(history.history['acc'], 'blue')
plt.plot(history.history['val_acc'], 'orange')
plt.title("Model Accuracyt")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')

In [ ]:
print("VGG -Validation Loss: ", history.history['val_loss'][-1]*100)
print("VGG - Validation Accuracy: ", history.history['val_acc'][-1]*100)
print("\n")
print("VGG - Training Loss: ", history.history['loss'][-1]*100)
print("VGG - Training Accuracy: ", history.history['acc'][-1]*100)
print("\n")

In [ ]:
# Plotting Model Loss
plt.plot(history.history['loss'], 'blue')
plt.plot(history.history['val_loss'], 'orange')
plt.title("Model Loss for VGGNet")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validate'], loc='upper left')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Saving the model
import h5py
from keras.models import load_model
model.save('/content/gdrive/My Drive/results/VGG 16/caltech101/model_vgg19-caltech101.h5')

# Report

In [ ]:
# Evaluation Metrics
target_names = []
for i in range(101):
    a = 'Object '
    b = str(i)
    c = a+b
    c = [i]
    target_names.append((a+b))

def reports(X_test,y_test):
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)

    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss = score[0]*100
    Test_accuracy = score[1]*100
    kc=cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    #mse=mean_squared_error(y_test, y_pred)
    #mae=mean_absolute_error(y_test, y_pred)
    #precision=precision_score(y_test, y_pred, average='weighted')
    #print(classification_report(y_test, y_pred, target_names=target_names))


    
    return classification, confusion, Test_Loss, Test_accuracy ,kc#,mse,mae

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the Model
#model = load_model('/content/gdrive/My Drive/wangdataset/model_hb.h5')
from sklearn.metrics import classification_report, confusion_matrix,cohen_kappa_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score
# calculate result, loss, accuray and confusion matrix
classification, confusion, Test_loss, Test_accuracy,kc = reports(X_test,y_testHot)
classification = str(classification)
confusion_str = str(confusion)


In [ ]:
print("confusion matrix: ")
print('{}'.format(confusion_str))
print("KAppa Coeefecient=",kc)
print('Test loss {} (%)'.format(Test_loss))
print('Test accuracy {} (%)'.format(Test_accuracy))
#print("Mean Squared error=",mse)
#print("Mean absolute error=",mae)
print(classification)

In [ ]:
# Plotting the Confusion Matrix
import matplotlib.pyplot as plt
%matplotlib inline
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.get_cmap("Blues")):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    Normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    if normalize:
        cm = Normalized
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(Normalized, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.4f' if normalize else 'd'
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        thresh = cm[i].max() / 2.
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


plt.figure(figsize=(45,45))
plot_confusion_matrix(confusion, classes=target_names, normalize=False, 
                      title='Confusion matrix, without normalization')
#plt.savefig("/content/gdrive/My Drive/wangdataset/result/confusion_matrix_without_normalization.svg")
plt.show()
plt.figure(figsize=(45,45))
plot_confusion_matrix(confusion, classes=target_names, normalize=True, 
                      title='Normalized confusion matrix')
#plt.savefig("/content/gdrive/My Drive/wangdataset/result/confusion_matrix_with_normalization.svg")
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("/content/gdrive/My Drive/wangdataset/model_accuracy_100.svg")
plt.show()

# summarize history for loss 
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid(True)
plt.legend(['train', 'test'], loc='upper left') 
plt.savefig("/content/gdrive/My Drive/wangdataset/model_loss_100.svg")
plt.show()

In [ ]:
C= confusion / confusion.astype(np.float).sum(axis=1)
print(confusion)
print(C)
C=C*100
print(C)
a = [0] * 100
for i in range(0,99):
  a[i]=C[i][i]
  

In [ ]:
pd.DataFrame(a).to_csv("/content/gdrive/My Drive/COIL 100/class_accuracy.csv")